<a href="https://colab.research.google.com/github/rajeshsahu09/CS69002_9A_18CS60R19/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Sentiment Analysis

# Run on **GPU**



### Import Header Files

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import time

### Load the Dataset and Visualise

In [0]:
# from google.colab import files
# uploaded = files.upload()

# import io
# df_train = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
# df_train.head()
url = "https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Train_20K.csv"
df = pd.read_csv(url, sep='\t')

In [34]:
df.shape

(17999, 2)

In [0]:
df_1 = df[df['label']==1] # get only label == 1
df_0 = df[df['label']==0] # get only label == 0

range_1 = int(0.9*len(df_1))
range_2 = int(0.9*len(df_0))

np.random.seed(42) # fix the seed value
df_1 = df_1.iloc[np.random.permutation(len(df_1))] # shuffled the data set of label==1
df_0 = df_0.iloc[np.random.permutation(len(df_0))] # shuffled the data set of label==0

temp_1_train = df_1.iloc[:range_1]
temp_2_train = df_0.iloc[:range_2]
df_train = pd.concat([temp_1_train, temp_2_train])

temp_1_val = df_1.iloc[range_1:]
temp_2_val = df_0.iloc[range_2:]
df_val = pd.concat([temp_1_val, temp_2_val])

In [36]:
df_train = df_train.iloc[np.random.permutation(len(df_train))] # shuffled the data set of label==1
df_val = df_val.iloc[np.random.permutation(len(df_val))] # shuffled the data set of label==1
len(df_train), len(df_val)

(16198, 1801)

In [37]:
print('Number of Negative movie reviews', len(df_train[df_train['label']==0]))
print('Number of Positive movie reviews', len(df_train[df_train['label']==1]))
print('Number of movie reviews', len(df_train['label']))

Number of Negative movie reviews 8094
Number of Positive movie reviews 8104
Number of movie reviews 16198


In [38]:
print('Number of Negative movie reviews', len(df_val[df_val['label']==0]))
print('Number of Positive movie reviews', len(df_val[df_val['label']==1]))
print('Number of movie reviews', len(df_val['label']))

Number of Negative movie reviews 900
Number of Positive movie reviews 901
Number of movie reviews 1801


In [0]:
url = "https://raw.githubusercontent.com/binny-mathew/IITKGP_CS69002_Spring_2019/master/Dataset/Test_5K.csv"
df_test = pd.read_csv(url, sep='\t')

## Data pre-processing

### Get Train Data

In [0]:
train_raw_text_reviews = df_train['text'].astype(str).tolist()
train_text_labels = df_train['label'].astype(int).tolist()

###Get Validation Data

In [0]:
val_raw_text_reviews = df_val['text'].astype(str).tolist()
val_text_labels = df_val['label'].astype(int).tolist()

###Get Test Data

In [0]:
test_raw_text_reviews = df_test['text'].astype(str).tolist()
test_text_labels = df_test['label'].astype(int).tolist()

#### Cleaning the raw input data

In [43]:
import re
import string
# import nltk
# nltk.download("stopwords")
# from nltk.corpus import stopwords

# # Finding stop words
# stop_words = set(stopwords.words('english'))

import spacy
print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en_core_web_sm')
# stop word list
stop_words = set(spacy.lang.en.stop_words.STOP_WORDS)

spaCy Version: 2.0.18


In [0]:
def preprocess_document(doc):
    # negative sense should not be eleminated + some short representation
    CONTRACTIONS = {"didn't":"did not", "mayn't":"may not", "can't":"can not", "won't":"will not", "isn't":"is not", "amn't":"am not",\
                  "aren't":"are not", "wasn't":"was not", "weren't":"were not", "couldn't":"could not", \
                  "wouldn't":"would not", "don't":"do not", "doesn't":"does not",\
                  "i'll":"i will", "you'll":"you will", "they'll":"they will",\
                  "may've":"may have", "can've":"can have", "will've":"will have", "you've":"you have", \
                  "could've":"could have", "would've":"would have", "you've":"you have", "they":"they have",\
                  "i've":"i have", "you've":"you have", "we've":"we have", "there's":"there is", "i'm":"i am",\
                  "it's":"it is", "what's":"what is", "where's":"where is", "how's":"how is", "i'd":"i had"}
    punctuation = string.punctuation + "\n\n"
    punc_replace = ''.join([' ' for s in punctuation]) # required for replacing punctuation with null ('')
    doc_clean = doc.replace('-', ' ') # replace - with null str
    doc_clean = (doc_clean.encode('ascii', 'ignore')).decode("utf-8")
    doc_clean = doc_clean.replace('<br />', '') # replace <br /> with ''
    doc_clean = doc_clean.replace("’", "'") # replace <br /> with null str
    doc_clean = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in doc_clean.split(' ')] # replacing some common short forms
    doc_clean = " ".join(doc_clean) # list to sentence of strings
    doc_clean = re.sub(r'\W +', ' ', doc_clean) # except [a-zA-Z0-9_]
    doc_clean = re.sub(r'\d+', ' ', doc_clean) # remove numbers [0-9]
    trans_table = str.maketrans(punctuation, punc_replace); # replace punctuations with ' '
    doc_clean = ' '.join([word.translate(trans_table) for word in doc_clean.split(' ')])
    doc_clean = doc_clean.split(' ')
    doc_clean = [word for word in doc_clean if len(word) > 0]
    # removing the stopwords from a sentence
    doc_clean = [word for word in doc_clean if not word.lower() in stop_words or word.lower() == 'not' or word.lower() == 'no']
    return doc_clean

In [45]:
train_text_reviews = [preprocess_document(review.lower()) for review in train_raw_text_reviews]
print (train_text_reviews[len(train_text_reviews)-2])
print (train_text_labels[len(train_text_labels)-2])

['tales', 'told', 'film', 'shot', 'knowledge', 'combined', 'vignette', 'film', 'makers', 'relate', 'vignettes', 'having', 'connected', 'shrink', 'martin', 'kove', 'leads', 'kove', 'vignette', 'sexy', 'vivian', 'schilling', 'woman', 'afraid', 'sun', 'makes', 'adrian', 'monk', 'look', 'brave', 'having', 'paranoia', 'laced', 'evening', 'home', 'literally', 'scream', 'vivian', 'ridiculous', 'things', 'spends', 'majority', 'time', 'nighty', 'shows', 'amazing', 'features', 'film', 'worst', 'not', 'nail', 'biting', 'second', 'vignette', 'owned', 'bill', 'paxton', 'portrays', 'roommate', 'hell', 'geeky', 'roommate', 'allows', 'complete', 'advantage', 'bill', 'vignette', 'funny', 'man', 'fears', 'death', 'moment', 'like', 'pal', 'choked', 'death', 'olive', 'not', 'interesting', 'movie', 'chopped', 'little', 'thought', 'involved', 'bill', 'paxton', 'fans']
0


In [46]:
val_text_reviews = [preprocess_document(review.lower()) for review in val_raw_text_reviews]
print (val_text_reviews[len(val_text_reviews)-2])
print (val_text_labels[len(val_text_labels)-2])

['not', 'realize', 'sellers', 'poor', 'health', 'time', 'filming', 'passed', 'away', 'film', 's', 'release', 'fiendish', 'plot', 'dr', 'fu', 'manchu', 'painful', 'viewing', 'supposedly', 'lampoon', 'sax', 'rohmer', 's', 'famous', 'oriental', 'villain', 'lacks', 'focus', 'potential', 'satirical', 'commentary', 'anti', 'oriental', 'overtones', 'rohmer', 's', 'concept', 'ignored', 'movie', 'employs', 'racist', 'insults', 'hardly', 'actual', 'jokes', 'gags', 'actors', 'behaving', 'idiotically', 'spouting', 'dreary', 'lines', 'especially', 'distressing', 'sid', 'caesar', 'forced', 'spout', 'curses', 'racial', 'slurs', 'attempted', 'laughs', 'actors', 'embarrass', 'peter', 'sellers', 'plays', 'dual', 'roles', 'sinister', 'fu', 'manchu', 'trying', 'concoct', 'formula', 'regain', 'youth', 'stalwart', 'british', 'foe', 'nayland', 'smith', 'sellers', 'not', 'cent', 'bad', 'conveys', 'quirky', 'warmth', 'smith', 'discusses', 'fetishistic', 'attachment', 'lawn', 'mower', 's', 'oddly', 'moving', 'm

In [47]:
test_text_reviews = [preprocess_document(review.lower()) for review in test_raw_text_reviews]
print (test_text_reviews[len(test_text_reviews)-2])
print (test_text_labels[len(test_text_labels)-2])

['shining', 'know', 'weird', 'movie', 'movie', 'people', 'claim', 'not', 'like', 'horror', 'films', 'shining', 'terrific', 'film', 'stanley', 'kubrick', 's', 'classic', 'vision', 'stephen', 'king', 's', 'horror', 'tale', 'madness', 'blood', 'incredible', 'film', 'wither', 'seen', 'not', 'heard', 'know', 'lines', 'know', 'classic', 'images', 'forget', 'jack', 's', 's', 'johnny', 'forget', 'work', 'no', 'play', 'jack', 'dull', 'boy', 'forget', 'chilling', 'ending', 'film', 'unforgettable', 'honestly', 'opinion', 'kubrick', 's', 'best', 'work', 'know', 'lot', 'argument', 'department', 'lot', 'people', 'space', 'odyssey', 'clockwork', 'orange', 'dr', 'strangelove', 'film', 'pioneered', 'film', 'making', 'shining', 'perfected', 'tale', 'isolation', 'madness', 'terrifying', 'images', 'ultimate', 'ghost', 'story', 'crawl', 'underneath', 'skin', 'jack', 'torrance', 'jack', 's', 'son', 'danny', 'jack', 's', 'wife', 'wendy', 'arrive', 'overlook', 'hotel', 'closing', 'day', 'elderly', 'african', 

In [48]:
import gc
gc.collect()

51

In [0]:
def build_vocabulary(sentences):
    # Build vocabulary
    dictWordCount = {}
    for sent in sentences:
        for word in sent:
            dictWordCount[word] = 0 # initialising the dict value to zero
    for sent in sentences:
        for word in sent:
            dictWordCount[word] += 1 # updating the dictionary count
    
    temp = dictWordCount.copy()
    for key, val in temp.items():
        if(dictWordCount[key] <= 10):
            del dictWordCount[key]
    
    # Mapping from index to word
    vocabulary_inv = sorted(dictWordCount, key=dictWordCount.__getitem__, reverse=True)
    
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return vocabulary, vocabulary_inv

###Creating Tokens and Types

In [0]:
word_to_ix, ix_to_word = build_vocabulary(train_text_reviews+val_text_reviews+test_text_reviews)

In [51]:
VOCAB_SIZE = len(word_to_ix)
ix_to_word[word_to_ix['kick']]=='kick', word_to_ix['kick'], VOCAB_SIZE

(True, 1710, 17746)

In [52]:
gc.collect()

403

## PyTorch

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable # Automatic gradients are calculated and back-propagated through the computational graph
import copy
import csv
import time
from google.colab import files

## Generate the BOW Vectors

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix)).to('cuda:0') #, device=device) # make 1D vector of len = vocab size
    for word in sentence:
        if word not in word_to_ix:
#             raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            pass
        else:
            vec[word_to_ix[word]]+=1 # count the number of occurance of same word in a sentences
            
    return vec.view(1, -1)

### Prepare Input Train and Validation torch vector

In [55]:
# store the bag of word vectors for each sentences and wraping to tensor of torch type
tic = time.time()
train_data = [Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') for instance in train_text_reviews]
num_train_data = len(train_text_reviews)

val_data = [Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') for instance in val_text_reviews]
num_val_data = len(val_text_reviews)
toc = time.time()
num_train_data, num_val_data, (toc-tic)

(16198, 1801, 98.38699841499329)

### Prepare Test Instance

In [0]:
data_test = []
data_test = [Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') for instance in test_text_reviews]

### Validation Accuracy Computation

In [0]:
def evaluate_validation_accuracy(data, net):
    sum_loss = 0
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for i, instance in enumerate(data):
        label = val_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label
        
#         vec = Variable(make_bow_vector(instance, word_to_ix)).to('cuda:0') # wrap to tensor of torch type for instance
        
#         prob = net.forward(vec) # forward pass

        prob = net.forward(instance)
    
        _class = 1 if prob.item() > 0.5 else 0 # sigmoid activated
        
        loss = loss_function(prob, label) # compute the loss
        
        sum_loss += float(loss.item())
        
#         if(int(label) == _class and _class == 1):
#             true_positive += 1
            
#         if(int(label) == _class and _class == 0):
#             true_negative += 1
        
#         if(_class == 1 and int(label) == 0):
#             false_positive += 1
            
#         if(_class == 0 and int(label) == 1):
#             false_negative += 1
    
    
#     precision = float(true_positive) / (true_positive + false_positive)
#     recall = float(true_positive) / (true_positive + false_negative)
#     f_score = float(2)*precision*recall / (precision + recall)
#     accuracy = float(1)*(true_positive+true_negative)/(true_positive+true_negative+false_positive+false_negative)
    
    return float(sum_loss)/len(data)#, float(100)*accuracy, precision, recall, f_score

##Task-1

In [0]:
task1 = [(50, 'Task1A.mdl'), (100, 'Task1B.mdl'), (200, 'Task1C.mdl'), (500, 'Task1D.mdl')]

In [28]:
for num_of_hidden, task_name in task1:
    class BOWClassifier(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            super(BOWClassifier, self).__init__()
            SEED = 42
            torch.manual_seed(SEED)
            torch.cuda.manual_seed(SEED)
            self.hidden_size = hidden_size
            self.i2h = nn.Linear(input_size, hidden_size) # initialises weights and biases i2h
            self.h2o = nn.Linear(hidden_size, output_size) # initialises weights and biases h2o

        def forward(self, x):
            x = torch.relu(self.i2h(x)) # relu activation @ hidden layer
            x = torch.sigmoid(self.h2o(x)) # sigmoid activation @ output layer
            return x

    num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
    num_of_output = 1 # binary sentiment classes (+ve, -ve)

    bow = BOWClassifier(num_of_input, num_of_hidden, num_of_output).to('cuda:0') # initialises weights and biases
    
    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)
    
    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        
        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)
        
        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))
    
    torch.save(bow, task_name) ## Save the model
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))
    
    print ('\n')

Epoch 1/100
[On Training] ==> Time: 23.92s, Train Loss: 0.681506928, Train Accuracy: 60.14%
[On Validation] ==> Val loss: 0.666210379, Total Time: 23.92s
Epoch 2/100
[On Training] ==> Time: 24.28s, Train Loss: 0.650062438, Train Accuracy: 70.73%
[On Validation] ==> Val loss: 0.630566867, Total Time: 48.20s
Epoch 3/100
[On Training] ==> Time: 24.59s, Train Loss: 0.611577513, Train Accuracy: 74.47%
[On Validation] ==> Val loss: 0.590137132, Total Time: 72.79s
Epoch 4/100
[On Training] ==> Time: 23.74s, Train Loss: 0.568415287, Train Accuracy: 77.61%
[On Validation] ==> Val loss: 0.548192693, Total Time: 96.53s
Epoch 5/100
[On Training] ==> Time: 23.59s, Train Loss: 0.525151165, Train Accuracy: 80.02%
[On Validation] ==> Val loss: 0.509592303, Total Time: 120.12s
Epoch 6/100
[On Training] ==> Time: 23.96s, Train Loss: 0.486261342, Train Accuracy: 81.88%
[On Validation] ==> Val loss: 0.477335682, Total Time: 144.09s
Epoch 7/100
[On Training] ==> Time: 23.91s, Train Loss: 0.453430340, Train

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task1A.mdl
[On Testing] Time: 2.395127296447754, Test Accuracy: 87.38%


Epoch 1/100
[On Training] ==> Time: 26.19s, Train Loss: 0.681662652, Train Accuracy: 56.80%
[On Validation] ==> Val loss: 0.665698624, Total Time: 26.19s
Epoch 2/100
[On Training] ==> Time: 26.29s, Train Loss: 0.648235821, Train Accuracy: 70.39%
[On Validation] ==> Val loss: 0.627552664, Total Time: 52.48s
Epoch 3/100
[On Training] ==> Time: 26.17s, Train Loss: 0.606963007, Train Accuracy: 75.35%
[On Validation] ==> Val loss: 0.585346470, Total Time: 78.65s
Epoch 4/100
[On Training] ==> Time: 26.30s, Train Loss: 0.562110687, Train Accuracy: 78.37%
[On Validation] ==> Val loss: 0.542950579, Total Time: 104.94s
Epoch 5/100
[On Training] ==> Time: 27.93s, Train Loss: 0.518668633, Train Accuracy: 80.39%
[On Validation] ==> Val loss: 0.504989100, Total Time: 132.87s
Epoch 6/100
[On Training] ==> Time: 28.08s, Train Loss: 0.480584246, Train Accuracy: 82.00%
[On Validation] ==> Val loss: 0.4

In [0]:
files.download('Task1A.mdl')
files.download('Task1B.mdl')
files.download('Task1C.mdl')
files.download('Task1D.mdl')

##Task-2

In [0]:
task2 = [(10, 10, 'Task2A.mdl'), (20, 10, 'Task2B.mdl'), (30, 30, 'Task2C.mdl'), (50, 50, 'Task2D.mdl'), (100, 50, 'Task2E.mdl')]

In [59]:
for num_of_hidden1, num_of_hidden2, task_name in task2:
    class BOWClassifier(nn.Module):
        def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
            super(BOWClassifier, self).__init__()
            SEED = 42
            torch.manual_seed(SEED)
            torch.cuda.manual_seed(SEED)
            self.i2h = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h
            self.h2h = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h2h
            self.h2o = nn.Linear(hidden_size2, output_size) # initialises weights and biases h2o

        def forward(self, x):
            x = torch.relu(self.i2h(x)) # relu activation @ hidden layer
            x = torch.relu(self.h2h(x)) # relu activation @ hidden layer
            x = torch.sigmoid(self.h2o(x)) # sigmoid activation @ output layer
            return x

    num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
    num_of_output = 1 # binary sentiment classes (+ve, -ve)

    bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_output).to('cuda:0') # initialises weights and biases

    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        
        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)

    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))
    
    print("\n")

Epoch 1/100
[On Training] ==> Time: 26.86s, Train Loss: 0.689191229, Train Accuracy: 58.30%
[On Validation] ==> Val loss: 0.683963105, Total Time: 26.86s
Epoch 2/100
[On Training] ==> Time: 26.21s, Train Loss: 0.678321193, Train Accuracy: 66.12%
[On Validation] ==> Val loss: 0.672020574, Total Time: 53.07s
Epoch 3/100
[On Training] ==> Time: 26.62s, Train Loss: 0.664094258, Train Accuracy: 69.19%
[On Validation] ==> Val loss: 0.654753870, Total Time: 79.70s
Epoch 4/100
[On Training] ==> Time: 26.77s, Train Loss: 0.643077088, Train Accuracy: 72.42%
[On Validation] ==> Val loss: 0.629783991, Total Time: 106.47s
Epoch 5/100
[On Training] ==> Time: 27.25s, Train Loss: 0.613262530, Train Accuracy: 75.71%
[On Validation] ==> Val loss: 0.595665983, Total Time: 133.71s
Epoch 6/100
[On Training] ==> Time: 26.94s, Train Loss: 0.573307074, Train Accuracy: 78.87%
[On Validation] ==> Val loss: 0.552970880, Total Time: 160.66s
Epoch 7/100
[On Training] ==> Time: 26.39s, Train Loss: 0.525504909, Trai

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task2A.mdl
[On Testing] Time: 2.4614241123199463, Test Accuracy: 87.46%


Epoch 1/100
[On Training] ==> Time: 27.90s, Train Loss: 0.694110947, Train Accuracy: 49.98%
[On Validation] ==> Val loss: 0.689399113, Total Time: 27.90s
Epoch 2/100
[On Training] ==> Time: 28.00s, Train Loss: 0.684947238, Train Accuracy: 52.11%
[On Validation] ==> Val loss: 0.679449860, Total Time: 55.91s
Epoch 3/100
[On Training] ==> Time: 28.79s, Train Loss: 0.673846138, Train Accuracy: 66.24%
[On Validation] ==> Val loss: 0.666359857, Total Time: 84.70s
Epoch 4/100
[On Training] ==> Time: 27.79s, Train Loss: 0.657531419, Train Accuracy: 73.60%
[On Validation] ==> Val loss: 0.646202123, Total Time: 112.49s
Epoch 5/100
[On Training] ==> Time: 27.25s, Train Loss: 0.631880520, Train Accuracy: 75.31%
[On Validation] ==> Val loss: 0.615079856, Total Time: 139.74s
Epoch 6/100
[On Training] ==> Time: 27.70s, Train Loss: 0.592946047, Train Accuracy: 77.53%
[On Validation] ==> Val loss: 0.

In [0]:
files.download('Task2A.mdl')
files.download('Task2B.mdl')
files.download('Task2C.mdl')
files.download('Task2D.mdl')
files.download('Task2E.mdl')

##Task-3

In [0]:
task3 = [(100, 50, 10, 'Task3A.mdl'), (200, 100, 10, 'Task3B.mdl')]

In [62]:
for num_of_hidden1, num_of_hidden2, num_of_hidden3, task_name in task3:
    class BOWClassifier(nn.Module):
        def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
            super(BOWClassifier, self).__init__()
            SEED = 42
            torch.manual_seed(SEED)
            torch.cuda.manual_seed(SEED)
            self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
            self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
            self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
            self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

        def forward(self, x):
            x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
            x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
            x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
            x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
            return x

    num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
    num_of_output = 1 # binary sentiment classes (+ve, -ve)

    bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases

    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model

        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))
    
    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

Epoch 1/100
[On Training] ==> Time: 35.35s, Train Loss: 0.694936959, Train Accuracy: 49.97%
[On Validation] ==> Val loss: 0.693881880, Total Time: 35.35s
Epoch 2/100
[On Training] ==> Time: 34.67s, Train Loss: 0.693303970, Train Accuracy: 49.97%
[On Validation] ==> Val loss: 0.692671734, Total Time: 70.02s
Epoch 3/100
[On Training] ==> Time: 34.74s, Train Loss: 0.692250105, Train Accuracy: 49.97%
[On Validation] ==> Val loss: 0.691733172, Total Time: 104.76s
Epoch 4/100
[On Training] ==> Time: 34.85s, Train Loss: 0.691292826, Train Accuracy: 49.97%
[On Validation] ==> Val loss: 0.690753229, Total Time: 139.61s
Epoch 5/100
[On Training] ==> Time: 35.50s, Train Loss: 0.690191506, Train Accuracy: 50.54%
[On Validation] ==> Val loss: 0.689503467, Total Time: 175.12s
Epoch 6/100
[On Training] ==> Time: 35.95s, Train Loss: 0.688624498, Train Accuracy: 59.42%
[On Validation] ==> Val loss: 0.687499562, Total Time: 211.07s
Epoch 7/100
[On Training] ==> Time: 34.86s, Train Loss: 0.685902498, Tra

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task3A.mdl
[On Testing] Time: 2.857330083847046, Test Accuracy: 87.60%


Epoch 1/100
[On Training] ==> Time: 40.43s, Train Loss: 0.692922032, Train Accuracy: 50.02%
[On Validation] ==> Val loss: 0.692548030, Total Time: 40.43s
Epoch 2/100
[On Training] ==> Time: 41.46s, Train Loss: 0.692221851, Train Accuracy: 51.05%
[On Validation] ==> Val loss: 0.691717098, Total Time: 81.90s
Epoch 3/100
[On Training] ==> Time: 40.99s, Train Loss: 0.691286394, Train Accuracy: 57.38%
[On Validation] ==> Val loss: 0.690668257, Total Time: 122.89s
Epoch 4/100
[On Training] ==> Time: 41.74s, Train Loss: 0.690016294, Train Accuracy: 64.82%
[On Validation] ==> Val loss: 0.689124106, Total Time: 164.63s
Epoch 5/100
[On Training] ==> Time: 42.47s, Train Loss: 0.688054048, Train Accuracy: 67.40%
[On Validation] ==> Val loss: 0.686661194, Total Time: 207.10s
Epoch 6/100
[On Training] ==> Time: 42.08s, Train Loss: 0.685037139, Train Accuracy: 68.07%
[On Validation] ==> Val loss: 0.

In [0]:
files.download('Task3A.mdl')
files.download('Task3B.mdl')

## TASK-4

In [0]:
task4 = [(30, 20, 10, 'Task4A.mdl'), (100, 100, 0, 'Task4B.mdl'), (100, 10, 0, 'Task4C.mdl')]

In [65]:
k = 1
for num_of_hidden1, num_of_hidden2, num_of_hidden3, task_name in task4:
    if(k == 1):
        k = 2
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
    elif(k == 2):
        k = 2
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h2h = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases i2h
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h2o = nn.Linear(hidden_size2, output_size) # initialises weights and biases h2o

            def forward(self, x):
                x = torch.relu(self.i2h(x)) # relu activation @ hidden layer
                x = torch.relu(self.h2h(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h2o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_output).to('cuda:0') # initialises weights and biases

    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
    
        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

Epoch 1/100
[On Training] ==> Time: 33.35s, Train Loss: 0.694767215, Train Accuracy: 50.03%
[On Validation] ==> Val loss: 0.693907171, Total Time: 33.35s
Epoch 2/100
[On Training] ==> Time: 33.38s, Train Loss: 0.693389675, Train Accuracy: 50.03%
[On Validation] ==> Val loss: 0.692973473, Total Time: 66.73s
Epoch 3/100
[On Training] ==> Time: 33.82s, Train Loss: 0.692659883, Train Accuracy: 50.04%
[On Validation] ==> Val loss: 0.692368075, Total Time: 100.55s
Epoch 4/100
[On Training] ==> Time: 33.79s, Train Loss: 0.692084617, Train Accuracy: 51.22%
[On Validation] ==> Val loss: 0.691792813, Total Time: 134.34s
Epoch 5/100
[On Training] ==> Time: 32.06s, Train Loss: 0.691481333, Train Accuracy: 56.62%
[On Validation] ==> Val loss: 0.691124821, Total Time: 166.40s
Epoch 6/100
[On Training] ==> Time: 31.25s, Train Loss: 0.690742162, Train Accuracy: 62.57%
[On Validation] ==> Val loss: 0.690270678, Total Time: 197.65s
Epoch 7/100
[On Training] ==> Time: 31.17s, Train Loss: 0.689755137, Tra

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task4A.mdl
[On Testing] Time: 2.9076828956604004, Test Accuracy: 87.52%


Epoch 1/100
[On Training] ==> Time: 31.86s, Train Loss: 0.691614258, Train Accuracy: 49.98%
[On Validation] ==> Val loss: 0.688385615, Total Time: 31.86s
Epoch 2/100
[On Training] ==> Time: 31.30s, Train Loss: 0.683821628, Train Accuracy: 55.48%
[On Validation] ==> Val loss: 0.678329603, Total Time: 63.16s
Epoch 3/100
[On Training] ==> Time: 31.64s, Train Loss: 0.670527282, Train Accuracy: 68.45%
[On Validation] ==> Val loss: 0.661184870, Total Time: 94.79s
Epoch 4/100
[On Training] ==> Time: 31.72s, Train Loss: 0.648115321, Train Accuracy: 72.71%
[On Validation] ==> Val loss: 0.633014937, Total Time: 126.51s
Epoch 5/100
[On Training] ==> Time: 31.77s, Train Loss: 0.612384750, Train Accuracy: 75.70%
[On Validation] ==> Val loss: 0.590897451, Total Time: 158.28s
Epoch 6/100
[On Training] ==> Time: 31.24s, Train Loss: 0.562413207, Train Accuracy: 78.64%
[On Validation] ==> Val loss: 0.

In [0]:
files.download('Task4A.mdl')
files.download('Task4B.mdl')
files.download('Task4C.mdl')

## Task-5

In [0]:
task5 = [('relu', 'Task5A.mdl'), ('tanh', 'Task5B.mdl'), ('sigmoid', 'Task5C.mdl')]

In [69]:
for activation, task_name in task5:
    if(activation == 'relu'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
    if(activation == 'tanh'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.tanh(self.i2h1(x)) # tanh activation @ hidden layer
                x = torch.tanh(self.h12h2(x)) # tanh activation @ hidden layer
                x = torch.tanh(self.h22h3(x)) # tanh activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
    if(activation == 'sigmoid'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.sigmoid(self.i2h1(x)) # sigmoid activation @ hidden layer
                x = torch.sigmoid(self.h12h2(x)) # sigmoid activation @ hidden layer
                x = torch.sigmoid(self.h22h3(x)) # sigmoid activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
    # define a loss function and an optimizer
    loss_function = nn.BCELoss()
    if(activation == 'sigmoid'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.001) ## sigmoid activation converges slowly
    else:
        opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)

    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model

        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

Epoch 1/100
[On Training] ==> Time: 29.70s, Train Loss: 0.694767215, Train Accuracy: 50.03%
[On Validation] ==> Val loss: 0.693907171, Total Time: 29.70s
Epoch 2/100
[On Training] ==> Time: 30.03s, Train Loss: 0.693389675, Train Accuracy: 50.03%
[On Validation] ==> Val loss: 0.692973473, Total Time: 59.73s
Epoch 3/100
[On Training] ==> Time: 29.63s, Train Loss: 0.692659883, Train Accuracy: 50.04%
[On Validation] ==> Val loss: 0.692368075, Total Time: 89.36s
Epoch 4/100
[On Training] ==> Time: 30.53s, Train Loss: 0.692084617, Train Accuracy: 51.22%
[On Validation] ==> Val loss: 0.691792813, Total Time: 119.89s
Epoch 5/100
[On Training] ==> Time: 30.67s, Train Loss: 0.691481333, Train Accuracy: 56.62%
[On Validation] ==> Val loss: 0.691124821, Total Time: 150.56s
Epoch 6/100
[On Training] ==> Time: 30.41s, Train Loss: 0.690742162, Train Accuracy: 62.57%
[On Validation] ==> Val loss: 0.690270678, Total Time: 180.97s
Epoch 7/100
[On Training] ==> Time: 29.96s, Train Loss: 0.689755137, Trai

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task5A.mdl
[On Testing] Time: 2.761873722076416, Test Accuracy: 87.52%


Epoch 1/100
[On Training] ==> Time: 30.44s, Train Loss: 0.691333151, Train Accuracy: 52.21%
[On Validation] ==> Val loss: 0.688294692, Total Time: 30.44s
Epoch 2/100
[On Training] ==> Time: 30.01s, Train Loss: 0.685687966, Train Accuracy: 63.18%
[On Validation] ==> Val loss: 0.682286099, Total Time: 60.45s
Epoch 3/100
[On Training] ==> Time: 29.92s, Train Loss: 0.678707658, Train Accuracy: 67.53%
[On Validation] ==> Val loss: 0.673533288, Total Time: 90.37s
Epoch 4/100
[On Training] ==> Time: 31.10s, Train Loss: 0.667702568, Train Accuracy: 69.17%
[On Validation] ==> Val loss: 0.659153392, Total Time: 121.47s
Epoch 5/100
[On Training] ==> Time: 30.06s, Train Loss: 0.649059623, Train Accuracy: 70.87%
[On Validation] ==> Val loss: 0.635048339, Total Time: 151.52s
Epoch 6/100
[On Training] ==> Time: 31.60s, Train Loss: 0.617592714, Train Accuracy: 73.31%
[On Validation] ==> Val loss: 0.5

In [0]:
files.download('Task5A.mdl')
files.download('Task5B.mdl')
files.download('Task5C.mdl')

##Task-6

In [0]:
def bigram_build_vocabulary(sentences):
    # Build bigram vocabulary
    bigrams_dictWordCount = {}
    bigrams_dictWordCount = {word+' '+sent.split()[i+1]:int(0) for sent in sentences  for i, word in enumerate(sent.split()) if i < len(sent.split())-1}

    for word in sent:
        dictWordCount[word] = 0 # initialising the dict value to zero
    for word in sent:
        dictWordCount[word] += 1 # updating the dictionary count
    
    temp = dictWordCount.copy()
    for key, val in temp.items():
        if(dictWordCount[key] <= 10):
            del dictWordCount[key]
    
    # Mapping from index to word
    vocabulary_inv = sorted(dictWordCount, key=dictWordCount.__getitem__, reverse=True)
    
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return vocabulary, vocabulary_inv

In [0]:
bigram_word_to_ix, bigram_ix_to_word = build_vocabulary(train_text_reviews+val_text_reviews+test_text_reviews)

In [0]:
BIGRAM_VOCAB_SIZE = len(bigram_word_to_ix)
bigram_ix_to_word[bigram_word_to_ix['kick']]=='kick', bigram_word_to_ix['kick'], BIGRAM_VOCAB_SIZE

In [0]:
gc.collect()

In [0]:
def bigram_make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix)).to('cuda:0') #, device=device) # make 1D vector of len = vocab size
    for word in sentence:
        if word not in word_to_ix:
#             raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            pass
        else:
            vec[word_to_ix[word]]+=1 # count the number of occurance of same word in a sentences
            
    return vec.view(1, -1)

##Task-7

In [0]:
task7 = [('ce', 'Task7A.mdl'), ('mse', 'Task7B.mdl'), ('hinge', 'Task7C.mdl')]

In [71]:
for loss_fun, task_name in task7:
    if(loss_fun == 'ce'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
        # define a loss function and 
        loss_function = nn.BCELoss()
        
    if(loss_fun == 'mse'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
        # define a loss function and 
        loss_function = nn.MSELoss()
        
    if(loss_fun == 'hinge'):
        class BOWClassifier(nn.Module):
            def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
                super(BOWClassifier, self).__init__()
                SEED = 42
                torch.manual_seed(SEED)
                torch.cuda.manual_seed(SEED)
                self.i2h1 = nn.Linear(input_size, hidden_size1) # initialises weights and biases i2h1
                self.drop1 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h12h2 = nn.Linear(hidden_size1, hidden_size2) # initialises weights and biases h12h2
                self.drop2 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h22h3 = nn.Linear(hidden_size2, hidden_size3) # initialises weights and biases h22h3
                self.drop3 = nn.Dropout(0.5) # Adding dropout layer
                
                self.h32o = nn.Linear(hidden_size3, output_size) # initialises weights and biases h32o

            def forward(self, x):
                x = torch.relu(self.i2h1(x)) # relu activation @ hidden layer
                x = torch.relu(self.h12h2(x)) # relu activation @ hidden layer
                x = torch.relu(self.h22h3(x)) # relu activation @ hidden layer
                x = torch.sigmoid(self.h32o(x)) # sigmoid activation @ output layer
                return x

        num_of_input = VOCAB_SIZE # Bag word restriction has to be equal to vocabulary
        num_of_hidden1 = 30
        num_of_hidden2 = 20
        num_of_hidden3 = 10
        num_of_output = 1 # binary sentiment classes (+ve, -ve)

        bow = BOWClassifier(num_of_input, num_of_hidden1, num_of_hidden2, num_of_hidden3, num_of_output).to('cuda:0') # initialises weights and biases
        
        # define a loss function and 
        loss_function = nn.HingeEmbeddingLoss()
        
    # define an optimizer
    if(loss_fun == 'ce'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.0001)
    elif(loss_fun == 'mse'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.0003)
    elif(loss_fun == 'hinge'):
        opt = torch.optim.SGD(bow.parameters(), lr = 0.01)
    gc.collect()

    # Train The Model
    epochs = 100
    # the training loop
    total_time = 0.0
    prev_val_loss = float('inf')
    val_loss = 0
    early_stop_bow = 0

    for e in range(epochs):
        tic = time.time() # start the timer
        correct = 0
        cumulative_loss = 0
        incorrect = 0

        for i, instance in enumerate(train_data): # train_text_reviews 
            # get the training data
            label = train_text_labels[i] # get the label of the corresponding instace
            label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

            bow.zero_grad() # reset the gradient for each instance

            pred = bow.forward(instance)

            loss = loss_function(pred, label) # compute the loss
            loss.backward() # backprop the loss
            opt.step() # performs parameter updation based on the current gradient

            cumulative_loss += float(loss.item()) # accumulate the loss over whole training sample

            pred_class = 1 if pred.item() > 0.5 else 0 # as sigmoid activated
            if(int(label) == pred_class): # counting correct prediction in each epoch
                correct += 1

        train_loss = float(cumulative_loss)/num_train_data
        train_accuracy = correct*float(100)/num_train_data

#         val_loss, val_accuracy, p, r, f = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model
        val_loss = evaluate_validation_accuracy(val_data, bow) # test val-data-set on currently trained model

        if(prev_val_loss-val_loss < 0.0): # early stop if nearly no change
            bow = copy.deepcopy(early_stop_bow)
            break

        early_stop_bow = copy.deepcopy(bow)

        prev_val_loss = val_loss

        toc = time.time() # final time
        total_time += (toc-tic)

        print("Epoch {}/{}\n[On Training] ==> Time: {:.2f}s, Train Loss: {:.9f}, Train Accuracy: {:.2f}%".format(e+1, epochs, (toc-tic), train_loss, train_accuracy))
#         print("[On Validation] ==> Precision: {:.3f}, Recall: {:.3f}, F-Score: {:.3f}, Val loss: {:.9f}, Val Accuracy: {:.2f}% Total Time: {:.2f}s".format(p, r, f, val_loss, val_accuracy, total_time))
        print("[On Validation] ==> Val loss: {:.9f}, Total Time: {:.2f}s".format(val_loss, total_time))

    torch.save(bow, task_name)
    
    print('--- AFTER TRAINING ---\tModel: {}'.format(task_name))
    correct = 0
    tic = time.time()
    for i, instance in enumerate(data_test):
        label = test_text_labels[i] # get the label of the corresponding instace
        label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label

        pred = bow.forward(instance)
        pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated

        if(int(label) == pred_class):
            correct += 1
    toc = time.time()
    print("[On Testing] Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

    print("\n")

Epoch 1/100
[On Training] ==> Time: 31.21s, Train Loss: 0.694767215, Train Accuracy: 50.03%
[On Validation] ==> Val loss: 0.693907171, Total Time: 31.21s
Epoch 2/100
[On Training] ==> Time: 31.66s, Train Loss: 0.693389675, Train Accuracy: 50.03%
[On Validation] ==> Val loss: 0.692973473, Total Time: 62.87s
Epoch 3/100
[On Training] ==> Time: 32.71s, Train Loss: 0.692659883, Train Accuracy: 50.04%
[On Validation] ==> Val loss: 0.692368075, Total Time: 95.59s
Epoch 4/100
[On Training] ==> Time: 32.01s, Train Loss: 0.692084617, Train Accuracy: 51.22%
[On Validation] ==> Val loss: 0.691792813, Total Time: 127.60s
Epoch 5/100
[On Training] ==> Time: 32.12s, Train Loss: 0.691481333, Train Accuracy: 56.62%
[On Validation] ==> Val loss: 0.691124821, Total Time: 159.72s
Epoch 6/100
[On Training] ==> Time: 30.84s, Train Loss: 0.690742162, Train Accuracy: 62.57%
[On Validation] ==> Val loss: 0.690270678, Total Time: 190.57s
Epoch 7/100
[On Training] ==> Time: 31.08s, Train Loss: 0.689755137, Trai

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type BOWClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


--- AFTER TRAINING ---	Model: Task7A.mdl
[On Testing] Time: 2.860544443130493, Test Accuracy: 87.52%


Epoch 1/100
[On Training] ==> Time: 31.02s, Train Loss: 0.250621006, Train Accuracy: 50.03%
[On Validation] ==> Val loss: 0.250113133, Total Time: 31.02s
Epoch 2/100
[On Training] ==> Time: 30.89s, Train Loss: 0.249846335, Train Accuracy: 50.04%
[On Validation] ==> Val loss: 0.249615604, Total Time: 61.91s
Epoch 3/100
[On Training] ==> Time: 31.00s, Train Loss: 0.249405547, Train Accuracy: 52.49%
[On Validation] ==> Val loss: 0.249171863, Total Time: 92.91s
Epoch 4/100
[On Training] ==> Time: 31.53s, Train Loss: 0.248906647, Train Accuracy: 61.12%
[On Validation] ==> Val loss: 0.248571691, Total Time: 124.43s
Epoch 5/100
[On Training] ==> Time: 30.82s, Train Loss: 0.248160170, Train Accuracy: 66.38%
[On Validation] ==> Val loss: 0.247620425, Total Time: 155.26s
Epoch 6/100
[On Training] ==> Time: 30.96s, Train Loss: 0.246818637, Train Accuracy: 69.01%
[On Validation] ==> Val loss: 0.2

In [0]:
files.download('Task7A.mdl')
files.download('Task7B.mdl')
files.download('Task7C.mdl')

In [0]:
# import matplotlib.pyplot as plt
# plt.figure("Image")
# plt.title("Loss vs Epoch")
# val_loss_history_plt =  [float(i)/sum(val_loss_history) for i in val_loss_history] # normalised between 0-1
# train_loss_history_plt =  [float(i)/sum(train_loss_history) for i in train_loss_history] # normalised between 0-1
# f_score_plt = [float(i)/sum(f_score) for i in f_score] # normalised between 0-1
# plt.plot(val_loss_history_plt, c="red", label="Validation Loss")
# plt.plot(train_loss_history_plt, c="blue", label = "Training Loss")
# plt.plot(f_score_plt, c="green", label = "F-Score")
# plt.legend()

In [0]:
from google.colab import files
temp_test = files.upload()

Saving Task3A.mdl to Task3A.mdl


In [0]:
# _model_ = torch.load(io.BytesIO(temp_test['Task2C.mdl']))
_model_ = torch.load('Task5C.mdl')

In [0]:
# for params in _model_.parameters():
#     print (params)

In [0]:
print('--- AFTER TRAINING ---')
correct = 0
tic = time.time()
for i, instance in enumerate(data_test):
    label = test_text_labels[i] # get the label of the corresponding instace
    label = Variable(torch.FloatTensor([label])).resize_((1,1)).to('cuda:0') # wraps a tensor for label
    
    pred = _model_.forward(instance)
    pred_class = 1 if pred.item() > 0.5 else 0 # sigmoid activated
    
    if(int(label) == pred_class):
        correct += 1
toc = time.time()
print("Time: {}, Test Accuracy: {:.2f}%".format((toc-tic), correct*100/len(data_test)))

--- AFTER TRAINING ---
Time: 2.400538921356201, Test Accuracy: 87.58%


In [0]:
# predict on test dataset
# with open("Task1A.csv", "wb") as _f:
#     writer = csv.writer(_f)
#     writer.writerows(a)

In [0]:
np.save('vocab.npy', word_to_ix)

from google.colab import files
files.download("vocab.npy")